In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Subset

import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [2]:
#feature extraction indexing for classes
def select_class_index(labels, n):
    labels = np.array(labels)
    selected = []

    for c in range(10):
        idx = np.where(labels == c)[0][:n]
        selected.extend(idx)

    return selected

In [3]:
# load in cifar-10 dataset
train_set = torchvision.datasets.CIFAR10(root="F:/CIFAR10_Project/data", train=True, download=True)
test_set = torchvision.datasets.CIFAR10(root="F:/CIFAR10_Project/data", train=False, download=True)

# select the first 500 train images per class
train_indices = select_class_index(train_set.targets, 500)
train_subset = torch.utils.data.Subset(train_set, train_indices)

# select the first 100 test images
test_indices = select_class_index(test_set.targets, 100)
test_subset = torch.utils.data.Subset(test_set, test_indices)

print(len(train_subset))  # Should be 500 * 10 = 5000
print(len(test_subset))   # Should be 1000

print(len(train_indices))  # 5000
print(set([train_set.targets[i] for i in train_indices])) # check how many classes ( should be 0-9 )

5000
1000
5000
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [4]:
#resize to 224x224x3
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
# use dataloaders to hold the resized images

train_subset.dataset.transform = transform_resnet
test_subset.dataset.transform = transform_resnet

train_loader = DataLoader(train_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

In [6]:
# load RestNet-18 and remove the last layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.DEFAULT
resnet = resnet18(weights=weights)
resnet.fc = nn.Identity() # last layer removed
resnet = resnet.to(device)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# resnet-18 feature extractor
#def extract_features(dataset):
   # loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)
def extract_features(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    feats = []
    labels = []

    with torch.no_grad():
        for images, lbls in loader:
            images = images.to(device)
            f = resnet(images)
            feats.append(f.cpu())
            labels.append(lbls)

    return torch.cat(feats), torch.cat(labels)

In [8]:
#train_features, train_labels = extract_features(train_subset.dataset)
#test_features, test_labels = extract_features(test_subset.dataset)
train_features, train_labels = extract_features(train_subset)
test_features, test_labels = extract_features(test_subset)

In [9]:
# TypeError: can't convert cuda:0 device type tensor to numpy. 
# Use Tensor.cpu() to copy the tensor to host memory first.

train_features_np = train_features.cpu().numpy()
test_features_np = test_features.cpu().numpy()

pca = PCA(n_components=50)
train_pca = pca.fit_transform(train_features_np)
test_pca  = pca.transform(test_features_np)

print(train_pca.shape, test_pca.shape)

(5000, 50) (1000, 50)


In [10]:
#- Linear(50, 512)- ReLU- Linear(512, 512)- BatchNorm(512)- ReLU- Linear(512, 10)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(50, 512),
            nn.ReLU(),

            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.net(x)

In [11]:
# 2 layer mlp variant

class MLP_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(50, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        return self.net(x)

In [12]:
# 4 layer mlp variant

class MLP_4(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(50, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        return self.net(x)

In [13]:
#pytorch tensors

X_train = torch.tensor(train_pca, dtype=torch.float32)
y_train = torch.tensor(train_labels, dtype=torch.long)

X_test = torch.tensor(test_pca, dtype=torch.float32)
y_test = torch.tensor(test_labels, dtype=torch.long)

C:\Users\Max\AppData\Local\Temp\ipykernel_7708\4204448965.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(train_labels, dtype=torch.long)
C:\Users\Max\AppData\Local\Temp\ipykernel_7708\4204448965.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(test_labels, dtype=torch.long)


In [14]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset  = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [15]:
# You should use the cross-entropy loss torch.nn.CrossEntropyLoss for training. 
# Also, use the SGD optimizer with momentum=0.9.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [16]:
#training model

def train(model, loader):
    model.train()
    total_loss = 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        #optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(loader)

In [17]:
#collects predicted and true labels

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)

            out = model(X)
            preds = torch.argmax(out, dim=1)

            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

In [18]:
def evaluation_loop_full(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            preds = torch.argmax(out, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    return all_labels, all_preds

In [19]:
#actual training

for epoch in range(20):  # 20 is apparently good for 5k samples
    train_loss = train(model, train_loader)
    acc = evaluate(model, test_loader)

    print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | Test Accuracy: {acc*100:.2f}%")

Epoch 01 | Loss: 0.7596 | Test Accuracy: 81.30%
Epoch 02 | Loss: 0.4129 | Test Accuracy: 82.70%
Epoch 03 | Loss: 0.3289 | Test Accuracy: 83.70%
Epoch 04 | Loss: 0.2673 | Test Accuracy: 83.50%
Epoch 05 | Loss: 0.2257 | Test Accuracy: 82.70%
Epoch 06 | Loss: 0.1737 | Test Accuracy: 83.10%
Epoch 07 | Loss: 0.1356 | Test Accuracy: 82.30%
Epoch 08 | Loss: 0.1153 | Test Accuracy: 83.10%
Epoch 09 | Loss: 0.0854 | Test Accuracy: 82.30%
Epoch 10 | Loss: 0.0662 | Test Accuracy: 82.30%
Epoch 11 | Loss: 0.0585 | Test Accuracy: 82.10%
Epoch 12 | Loss: 0.0431 | Test Accuracy: 82.80%
Epoch 13 | Loss: 0.0526 | Test Accuracy: 81.60%
Epoch 14 | Loss: 0.0463 | Test Accuracy: 82.30%
Epoch 15 | Loss: 0.0499 | Test Accuracy: 83.00%
Epoch 16 | Loss: 0.0318 | Test Accuracy: 82.10%
Epoch 17 | Loss: 0.0312 | Test Accuracy: 82.70%
Epoch 18 | Loss: 0.0229 | Test Accuracy: 82.70%
Epoch 19 | Loss: 0.0259 | Test Accuracy: 81.40%
Epoch 20 | Loss: 0.0280 | Test Accuracy: 82.10%


In [22]:
def evaluate_model(y_true, y_pred):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='macro'))
    print("Recall :", recall_score(y_true, y_pred, average='macro'))
    print("F1-score :", f1_score(y_true, y_pred, average='macro'))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

print("Evaluation of three-layer MLP:")
y_true, y_pred = evaluation_loop_full(model, test_loader, device)
evaluate_model(y_true, y_pred)

Evaluation of three-layer MLP:
Accuracy: 0.821
Precision: 0.8229314736500685
Recall : 0.8210000000000001
F1-score : 0.8206237678140541
Confusion Matrix:
 [[79  2  6  2  1  0  1  1  4  4]
 [ 3 90  0  1  0  0  0  0  1  5]
 [ 5  0 82  5  2  1  4  1  0  0]
 [ 0  0  3 72  1 13  6  4  1  0]
 [ 3  0  6  6 70  2  3 10  0  0]
 [ 1  1  6 13  3 71  3  2  0  0]
 [ 1  0  3  2  1  3 88  2  0  0]
 [ 0  0  1  3  3  3  0 90  0  0]
 [ 5  1  1  1  0  0  0  1 90  1]
 [ 2  6  0  1  1  0  0  0  1 89]]


In [34]:
# save 3-layer MLP model

torch.save(model, "MLP_3_saved.pth")

In [35]:
# instantiate and train 2-layer model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model2 = MLP_2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    train_loss = train(model2, train_loader)
    acc = evaluate(model2, test_loader)
    print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | Test Accuracy: {acc*100:.2f}%")

Epoch 01 | Loss: 0.7924 | Test Accuracy: 81.00%
Epoch 02 | Loss: 0.4473 | Test Accuracy: 82.70%
Epoch 03 | Loss: 0.3788 | Test Accuracy: 83.50%
Epoch 04 | Loss: 0.3477 | Test Accuracy: 83.30%
Epoch 05 | Loss: 0.3234 | Test Accuracy: 83.90%
Epoch 06 | Loss: 0.3054 | Test Accuracy: 83.20%
Epoch 07 | Loss: 0.2894 | Test Accuracy: 83.00%
Epoch 08 | Loss: 0.2623 | Test Accuracy: 83.80%
Epoch 09 | Loss: 0.2450 | Test Accuracy: 84.00%
Epoch 10 | Loss: 0.2260 | Test Accuracy: 83.50%
Epoch 11 | Loss: 0.2156 | Test Accuracy: 83.90%
Epoch 12 | Loss: 0.1984 | Test Accuracy: 83.60%
Epoch 13 | Loss: 0.1836 | Test Accuracy: 83.80%
Epoch 14 | Loss: 0.1722 | Test Accuracy: 83.70%
Epoch 15 | Loss: 0.1618 | Test Accuracy: 83.30%
Epoch 16 | Loss: 0.1494 | Test Accuracy: 83.20%
Epoch 17 | Loss: 0.1369 | Test Accuracy: 83.30%
Epoch 18 | Loss: 0.1306 | Test Accuracy: 83.10%
Epoch 19 | Loss: 0.1219 | Test Accuracy: 83.40%
Epoch 20 | Loss: 0.1088 | Test Accuracy: 82.90%


In [36]:
y_true, y_pred = evaluation_loop_full(model2, test_loader, device)
evaluate_model(y_true, y_pred)

Accuracy: 0.829
Precision: 0.8317982405345697
Recall : 0.829
F1-score : 0.8297482564498798
Confusion Matrix:
 [[85  1  4  1  0  0  1  2  3  3]
 [ 3 91  0  1  0  0  0  0  0  5]
 [ 3  0 75  5  3  4  9  1  0  0]
 [ 0  0  2 72  2 15  7  2  0  0]
 [ 2  0  5  7 76  3  1  5  1  0]
 [ 0  0  6 17  2 70  2  2  1  0]
 [ 1  0  4  3  1  3 87  1  0  0]
 [ 1  0  1  4  7  3  0 84  0  0]
 [ 5  0  1  0  0  0  0  1 93  0]
 [ 0  2  0  2  0  0  0  0  0 96]]


In [37]:
torch.save(model2, "MLP_2_saved.pth")

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model4 = MLP_4().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model4.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    train_loss = train(model4, train_loader)
    acc = evaluate(model4, test_loader)
    print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | Test Accuracy: {acc*100:.2f}%")

Epoch 01 | Loss: 0.7781 | Test Accuracy: 81.60%
Epoch 02 | Loss: 0.4003 | Test Accuracy: 82.70%
Epoch 03 | Loss: 0.2906 | Test Accuracy: 82.20%
Epoch 04 | Loss: 0.1922 | Test Accuracy: 82.00%
Epoch 05 | Loss: 0.1438 | Test Accuracy: 82.20%
Epoch 06 | Loss: 0.0881 | Test Accuracy: 83.10%
Epoch 07 | Loss: 0.0952 | Test Accuracy: 81.10%
Epoch 08 | Loss: 0.0902 | Test Accuracy: 83.10%
Epoch 09 | Loss: 0.0781 | Test Accuracy: 81.90%
Epoch 10 | Loss: 0.0593 | Test Accuracy: 82.20%
Epoch 11 | Loss: 0.0423 | Test Accuracy: 83.10%
Epoch 12 | Loss: 0.0510 | Test Accuracy: 82.60%
Epoch 13 | Loss: 0.0262 | Test Accuracy: 82.50%
Epoch 14 | Loss: 0.0301 | Test Accuracy: 82.90%
Epoch 15 | Loss: 0.0287 | Test Accuracy: 82.20%
Epoch 16 | Loss: 0.0954 | Test Accuracy: 81.90%
Epoch 17 | Loss: 0.0742 | Test Accuracy: 81.90%
Epoch 18 | Loss: 0.0912 | Test Accuracy: 82.00%
Epoch 19 | Loss: 0.0448 | Test Accuracy: 82.40%
Epoch 20 | Loss: 0.0183 | Test Accuracy: 82.00%


In [39]:
y_true, y_pred = evaluation_loop_full(model4, test_loader, device)
evaluate_model(y_true, y_pred)

Accuracy: 0.82
Precision: 0.8273689102003772
Recall : 0.8200000000000001
F1-score : 0.8204099254681589
Confusion Matrix:
 [[77  1  3  2  0  0  1  1 10  5]
 [ 1 89  0  1  1  0  1  0  1  6]
 [ 5  0 70 10  4  2  7  2  0  0]
 [ 1  0  2 81  3  5  6  1  1  0]
 [ 3  0  3  8 77  1  2  4  1  1]
 [ 0  0  6 20  1 68  2  2  1  0]
 [ 1  0  1  3  1  1 91  1  1  0]
 [ 0  0  0  4  8  4  0 84  0  0]
 [ 4  1  1  1  0  0  0  0 90  3]
 [ 2  1  1  1  0  0  1  0  1 93]]


In [40]:
torch.save(model4, "MLP_4_saved.pth")

In [41]:
#load models

model2_loaded = torch.load("MLP_2_saved.pth", weights_only=False, map_location=device)
model2_loaded.eval()

y_true, y_pred = evaluation_loop_full(model2_loaded, test_loader, device)
evaluate_model(y_true, y_pred)


Accuracy: 0.829
Precision: 0.8317982405345697
Recall : 0.829
F1-score : 0.8297482564498798
Confusion Matrix:
 [[85  1  4  1  0  0  1  2  3  3]
 [ 3 91  0  1  0  0  0  0  0  5]
 [ 3  0 75  5  3  4  9  1  0  0]
 [ 0  0  2 72  2 15  7  2  0  0]
 [ 2  0  5  7 76  3  1  5  1  0]
 [ 0  0  6 17  2 70  2  2  1  0]
 [ 1  0  4  3  1  3 87  1  0  0]
 [ 1  0  1  4  7  3  0 84  0  0]
 [ 5  0  1  0  0  0  0  1 93  0]
 [ 0  2  0  2  0  0  0  0  0 96]]


In [42]:
 #load models

model3_loaded = torch.load("MLP_3_saved.pth", weights_only=False, map_location=device)
model3_loaded.eval()

y_true, y_pred = evaluation_loop_full(model3_loaded, test_loader, device)
evaluate_model(y_true, y_pred)


Accuracy: 0.821
Precision: 0.8229314736500685
Recall : 0.8210000000000001
F1-score : 0.8206237678140541
Confusion Matrix:
 [[79  2  6  2  1  0  1  1  4  4]
 [ 3 90  0  1  0  0  0  0  1  5]
 [ 5  0 82  5  2  1  4  1  0  0]
 [ 0  0  3 72  1 13  6  4  1  0]
 [ 3  0  6  6 70  2  3 10  0  0]
 [ 1  1  6 13  3 71  3  2  0  0]
 [ 1  0  3  2  1  3 88  2  0  0]
 [ 0  0  1  3  3  3  0 90  0  0]
 [ 5  1  1  1  0  0  0  1 90  1]
 [ 2  6  0  1  1  0  0  0  1 89]]


In [43]:
 #load models

model4_loaded = torch.load("MLP_4_saved.pth", weights_only=False, map_location=device)
model4_loaded.eval()

y_true, y_pred = evaluation_loop_full(model4_loaded, test_loader, device)
evaluate_model(y_true, y_pred)

Accuracy: 0.82
Precision: 0.8273689102003772
Recall : 0.8200000000000001
F1-score : 0.8204099254681589
Confusion Matrix:
 [[77  1  3  2  0  0  1  1 10  5]
 [ 1 89  0  1  1  0  1  0  1  6]
 [ 5  0 70 10  4  2  7  2  0  0]
 [ 1  0  2 81  3  5  6  1  1  0]
 [ 3  0  3  8 77  1  2  4  1  1]
 [ 0  0  6 20  1 68  2  2  1  0]
 [ 1  0  1  3  1  1 91  1  1  0]
 [ 0  0  0  4  8  4  0 84  0  0]
 [ 4  1  1  1  0  0  0  0 90  3]
 [ 2  1  1  1  0  0  1  0  1 93]]


In [ ]:
|